In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
transform_svhn = transforms.Compose(
    [
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.Resize((28, 28))
    ])

transform_mnist = transforms.Compose(
    [
    transforms.Grayscale(),
    transforms.ToTensor(),
    transforms.RandomRotation((-75, 75))
    ])

'''
SVNH
'''
train_data_svhn = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)

train_svhn, val_svhn = train_test_split(train_data_svhn, test_size=0.2, random_state=42)

train_loader_svhn = torch.utils.data.DataLoader(train_svhn, batch_size=64, shuffle=True, num_workers=2)
val_loader_svhn = torch.utils.data.DataLoader(val_svhn, batch_size=64, shuffle=True, num_workers=2)

test_data_svhn = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)
test_loader_svhn = torch.utils.data.DataLoader(test_data_svhn, batch_size=64, shuffle=False, num_workers=2)

'''
MNIST
'''
train_data_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)

train_mnist, val_mnist= train_test_split(train_data_mnist, test_size=0.2, random_state=42)

train_loader_mnist = torch.utils.data.DataLoader(train_mnist, batch_size=64, shuffle=True, num_workers=2)
val_loader_mnist = torch.utils.data.DataLoader(val_mnist, batch_size=64, shuffle=True, num_workers=2)

test_data_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)
test_loader_mnist = torch.utils.data.DataLoader(test_data_mnist, batch_size=64, shuffle=False, num_workers=2)



Using downloaded and verified file: ./data\train_32x32.mat
Using downloaded and verified file: ./data\test_32x32.mat


In [4]:
loaders_svhn = {
    'train': train_loader_svhn,
    'val': val_loader_svhn,
    'test': test_loader_svhn
}

loaders_mnist = {
    'train': train_loader_mnist,
    'val': val_loader_mnist,
    'test': test_loader_mnist
}

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        self.FC1 = nn.Linear(32 * 7 * 7, 850)
        self.ReLU = nn.LeakyReLU(0.1)
        self.out = nn.Linear(850, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.FC1(x)
        x = self.ReLU(x)
        output = self.out(x)
        return output

In [6]:
cnn_mnist = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_mnist.parameters(), lr = 0.0001)

In [7]:
num_epochs = 2

def train(num_epochs, cnn, loaders):
    cnn.train()
    running_loss = 0.0
    total_step = len(loaders['train'])
    for epoch in range(num_epochs):
        for i, data in enumerate(loaders['train']):
            inputs, labels = data
            
            optimizer.zero_grad()
            
            outputs = cnn(inputs)
            loss = criterion(outputs, labels)
            loss.backward()             
            optimizer.step()                
            
            if (i+1) % 100 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

In [8]:
train(num_epochs, cnn_mnist, loaders_mnist)

Epoch 1/2, Loss: 1.2451661825180054
Epoch 1/2, Loss: 0.8758147358894348
Epoch 1/2, Loss: 0.7475005388259888
Epoch 1/2, Loss: 0.4989623427391052
Epoch 1/2, Loss: 0.6146449446678162
Epoch 1/2, Loss: 0.3575836718082428
Epoch 1/2, Loss: 0.4809367060661316
Epoch 2/2, Loss: 0.3329421281814575
Epoch 2/2, Loss: 0.4829939901828766
Epoch 2/2, Loss: 0.17274883389472961
Epoch 2/2, Loss: 0.2718576192855835
Epoch 2/2, Loss: 0.41746148467063904
Epoch 2/2, Loss: 0.12569142878055573
Epoch 2/2, Loss: 0.19346126914024353


In [9]:
torch.save(cnn_mnist.state_dict(), './cnn_mnist.pth')

In [10]:
cnn_mnist = CNN()
cnn_mnist.load_state_dict(torch.load('./cnn_mnist.pth'))
cnn_mnist.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (FC1): Linear(in_features=1568, out_features=850, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.1)
  (out): Linear(in_features=850, out_features=10, bias=True)
)

In [19]:
def test(model, loaders):
    true_labels=[]
    pred_labels=[]
    correct = 0
    total = 0
    losses = []
    with torch.no_grad():
        for i, data in enumerate(loaders['test'], 0):
            image, label = data
            pred = model(image)
            for i in range(len(data)):
                if torch.argmax(pred[i]) == label[i]:
                    correct += 1
                total += 1
    print(f"Acuracy: {round(correct/total*100, 2)}%")

In [20]:
test(cnn_mnist, loaders_mnist)

Acuracy: 90.13%


In [21]:
cnn_svhn = CNN()
cnn_svhn.load_state_dict(torch.load('./cnn_mnist.pth'))
cnn_svhn.eval()

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (FC1): Linear(in_features=1568, out_features=850, bias=True)
  (ReLU): LeakyReLU(negative_slope=0.1)
  (out): Linear(in_features=850, out_features=10, bias=True)
)

In [22]:
test(cnn_svhn, loaders_svhn)

Acuracy: 17.57%
